# Metadata

**Python Topics** - Fast API

# Conversation

**Human**

Help me create a fast api application from scratch.

**LLM**

Let's start small with only the fundamentals.
- **requirements.txt** for listing explicit dependency versions
- **main.py** which contains your application code

---
##### Setup the Environemnt
You only need 2 things at this stage
- Install initial dependencies `pip install fastapi uvicorn`
- Document the versions in requirements.txt as such
```
uvicorn==a.b.c
fastapi==x.y.z
```

##### Write your app
Here's a sample hello-world endpoint for you
```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def read_root():
    return "Hello World!"
```

##### Run your Application
```bash
uvicorn main:app --host 0.0.0.0 --port 8888
```

This will make your application go live on your system, if you navigate to http://localhost:8888 you should find a `Hello World!` message :D

**Human**

That's awesome! However, I just tried to change the message from hello world to something else, even though I saved, It does not change on the browser no matter how many times I refresh.

**LLM**

Yeah, we need to stop the server from the terminal and rerun it to make the changes reflect. A nice option in uvicorn is using the `--reload` flag as follows:

```bash
uvicorn main:app --host 0.0.0.0 --port 8888 --reload
```

This monitors your codebase, whenever you save any code files, It will automatically restart the app for you. It's a huge time saver.

**Human**

I also wanna set up a virtual environemnt

**LLM**

There are many types of virtual environments available... the simplest one is venv. 
Here's how to get started:
- Create the environemnt in the project's root dir `python -m venv venv`
- Activate it: `source venv/bin/activate` for linux & `venv\Scripts\activate` for windows.
- Install your dependencies `pip install -r requirements.txt`

And there you have it... a portable project via explicit & isolated dependencies! 

**Human**

I'm creating a create recipe form... a recipe should have a name, ingredients & preparation instrucitons... I want to do input validation

**LLM**

I think you will need `pydantic` for this. It works seamlessly with fastapi for handling input validations based on the types and models you create. 

First, Just `pip install pydantic` & add the version explicitly to your `requirements.txt`

Then, Let's define the pydantic model itself:
```python
from typing import List
from pydantic import BaseModel


class Ingredient(BasModel):
    name: str
    quantity: float
    unit: str


class CreateRecipe(BaseModel):
    name: str
    ingredients: List[Ingredient] 
    prep_instructions: str
```

**Human**

Please add the validation to the following function


```python
@app.post("/recipes/", status_code=status.HTTP_201_CREATED)
async def create_recipe(recipe):
    # Function doing some secret things.
    return {"message": "Recipe created successfully", "recipe": recipe}
```

**LLM**

This is exactly why `pydantic` shines... watch how we're adding the validation with minimal code changes.

```python
@app.post("/recipes/", status_code=status.HTTP_201_CREATED)
async def create_recipe(recipe: CreateRecipe):
    # Function doing some secret things.
    return {"message": "Recipe created successfully", "recipe": recipe}
```

Voala!

**Human**

What was the route for the auto generated docs?

**LLM**

It's `/docs`... [Here you go](http://localhost:8888/docs)

**Human**

Can you help me dockerize this? The secret code is just using pyodbc to connect to mssql 2017... that's the only additional dependency.

**LLM**

```Dockerfile
FROM python:3.9-slim

# Install ODBC Driver
RUN apt-get update && apt-get install --reinstall build-essential -y
RUN apt-get install curl -y
RUN curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
RUN apt-get update && apt-get install -y curl apt-transport-https \
    && curl https://packages.microsoft.com/config/debian/10/prod.list > /etc/apt/sources.list.d/mssql-release.list \
    && apt-get update \
    && ACCEPT_EULA=Y apt-get install -y msodbcsql17 unixodbc-dev
RUN ACCEPT_EULA=Y apt-get install -y --allow-unauthenticated mssql-tools
RUN echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
RUN echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bash_profile
RUN apt-get install -y unixodbc-dev

# Install dependencies
WORKDIR /app
COPY requirements.txt requirements
RUN pip install -r requirements.txt

# Copy Source Code
COPY . .

# Run Command
EXPOSE 80
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
```
---
Here are the considerations while creating this:
- Installing the `ODBC` driver and especially the msodbcsql17 was needed for your secret code to work.
- Installing the dependencies comes before the source code to take advantage of the docker layers and optimize build times.
- Running the app on port 80 internally as this is compatible with most cloud providers.


Please let me know if you have any questions.